In [2]:
import tarfile
import spacy
import re
import snorkel
import pandas as pd
import rubrix as rb
import numpy as np
import json
import sklearn
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from flashtext import KeywordProcessor

ModuleNotFoundError: No module named 'spacy'

In [39]:
data_scraped2 = pd.read_csv('D:\Downloads\Clean_data_Twitter_Emotion_Scraped_v5.csv')

In [2]:
data = pd.read_csv("D:\Downloads\Twitter_Emotion_Dataset_without_label.csv")
data_labelled = pd.read_csv("D:\Downloads\Twitter_Emotion_Dataset_with_label.csv")
data_scraped = pd.read_csv("D:\Downloads\Twitter_Data_Emotion_Scraped.csv")
dictionary = pd.read_csv("D:\Downloads\colloquial-indonesian-lexicon.csv")
data.head(100)

C:\Users\pradi\AppData\Local\Temp\ipykernel_37520\3795501739.py:3: DtypeWarning: Columns (9,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  data_scraped = pd.read_csv("D:\Downloads\Twitter_Data_Emotion_Scraped.csv")


,tweet
0,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu..."
1,"Sesama cewe lho (kayaknya), harusnya bisa lebi..."
2,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...
3,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng..."
4,"Sharing pengalaman aja, kemarin jam 18.00 bata..."
...,...
95,Mudah2an sudah terupload smua sebelum z mudik ...
96,Orang pendukung khilafah memang harus di black...
97,"jangan sok akrab ye mention mention gue , mali..."
98,"Alhamdulillah Prof, setelah berbicara semalam ..."


In [14]:
data_labelled['label'].count()


4401

In [9]:
data_labelled['label'] = data_labelled['label'].replace(['happy', 'anger', 'sadness', 'fear', 'love'], [0, 1, 2, 3, 4])


In [1]:
data_labelled = data_labelled.rename(columns={"tweet": "text"})
data_labelled_rb = rb.read_pandas(data_labelled, task="TextClassification")


NameError: name 'data_labelled' is not defined

In [11]:
data_scraped2 = data.rename(columns={"tweet": "text"})
data_scraped2_rb = rb.read_pandas(data, task="TextClassification")

2023-04-11 03:00:47.699 | WARNING  | rubrix.client.datasets:from_pandas:341 - Following columns are not supported by the TextClassificationRecord model and are ignored: ['tweet']


In [29]:
data_scraped = data_scraped.rename(columns={"tweet": "text"})
data_scraped_rb = rb.read_pandas(data_scraped, task="TextClassification")

In [38]:
training_data = data_labelled.sample(frac=0.8, random_state=42)
testing_data = data_labelled.drop(training_data.index)

print(f"Number of training examples: {training_data.shape[0]}")
print(f"Number of testing examples: {testing_data.shape[0]}")

Number of training examples: 3521
Number of testing examples: 880


In [14]:
data_scraped2 = data_scraped2.drop(columns=['Unnamed: 0'])

In [18]:
data_scraped2


,tweet
0,Sekarang udah ada union tapiiiiiiiii
1,people followed me and one person unfollowed m...
2,Sore yopp
3,Laper yaa mas Wkwwk
4,Oooh can wait
...,...
78215,Emang udah paling bener gausah cerita apa sm s...
78216,Merosotin cuy
78217,Lahh ternyata ada fans nya yang komen
78218,Employee commercial would rather QQ


In [40]:
import rubrix as rb
# build our training records without labels
records = [
    rb.TextClassificationRecord(
        text=row.tweet,
        metadata={"tweet":row.tweet},
    )
    for i,row in data_scraped2.iterrows()
]
rb.log(records, name="emotion_twitter_v10")

c:\Users\pradi\AppData\Local\Programs\Python\Python310\lib\site-packages\rubrix\client\models.py:43: UserWarning: Some metadata values exceed the max length. Those values will be truncated by keeping only the last 128 characters.
  warnings.warn(


  0%|          | 0/78220 [00:00<?, ?it/s]

78220 records logged to http://localhost:6900/datasets/rubrix/emotion_twitter_v10


BulkResponse(dataset='emotion_twitter_v10', processed=78220, failed=0)

In [41]:
# build our labelled records to evaluate our heuristic rules and optimize the thresholds
labels = ['happy', 'anger', 'sadness', 'fear', 'love']

records = [
    rb.TextClassificationRecord(
        text=row.tweet,
        annotation=labels[row.label],
        metadata={"Tweet":row.tweet},
    )
    for i,row in testing_data.iterrows()
]

# log the records to Rubrix
rb.log(records, name="emotion_twitter_v10")

  0%|          | 0/880 [00:00<?, ?it/s]

880 records logged to http://localhost:6900/datasets/rubrix/emotion_twitter_v10


BulkResponse(dataset='emotion_twitter_v10', processed=880, failed=0)

In [49]:
from rubrix.labeling.text_classification import WeakLabels

weak_labels = WeakLabels(dataset="emotion_twitter_v10")

Preparing rules:   0%|          | 0/27 [00:00<?, ?it/s]

Applying rules:   0%|          | 0/79100 [00:00<?, ?it/s]

In [50]:
weak_labels.summary()

,label,coverage,annotated_coverage,overlaps,conflicts,correct,incorrect,precision
sedih,{sadness},0.002617,0.012500,0.000544,0.000480,9,2,0.818182
senang,{happy},0.002554,0.000000,0.000177,0.000139,0,0,NaN
seneng,{happy},0.001871,0.006818,0.000303,0.000253,2,4,0.333333
marah,{anger},0.001669,0.012500,0.000303,0.000215,8,3,0.727273
takut,{fear},0.005247,0.075000,0.000430,0.000367,55,11,0.833333
serem,{fear},0.001264,0.022727,0.000126,0.000088,18,2,0.900000
sayang,{love},0.003805,0.097727,0.000721,0.000442,55,31,0.639535
cinta,{love},0.002857,0.071591,0.000493,0.000265,48,15,0.761905
astagfirullah,{anger},0.000632,0.001136,0.000051,0.000025,1,0,1.000000
alhamdulillah,{happy},0.004741,0.019318,0.000253,0.000139,14,3,0.823529


In [51]:
from rubrix.labeling.text_classification import Snorkel

label_model = Snorkel(weak_labels)

label_model.fit(lr=0.002, n_epochs=10, progress_bar=True)
print(label_model.score(output_str=True))

100%|██████████| 10/10 [00:00<00:00, 273.26epoch/s]

              precision    recall  f1-score   support

       happy       0.67      0.65      0.66        43
       anger       0.67      0.43      0.52        28
     sadness       0.65      0.28      0.39        39
        fear       0.85      0.89      0.87        80
        love       0.70      0.90      0.79       101

    accuracy                           0.73       291
   macro avg       0.71      0.63      0.65       291
weighted avg       0.72      0.73      0.71       291



In [49]:
weak_labels

In [23]:
data_labelled['label'] = data_labelled['label'].replace(['happy', 'anger', 'sadness', 'fear', 'love'],[0, 1, 2, 3, 4]  )
data_labelled

,label,tweet
0,1,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu..."
1,1,"Sesama cewe lho (kayaknya), harusnya bisa lebi..."
2,0,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...
3,1,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng..."
4,0,"Sharing pengalaman aja, kemarin jam 18.00 bata..."
...,...,...
4396,4,"Tahukah kamu, bahwa saat itu papa memejamkan m..."
4397,3,Sulitnya menetapkan Calon Wapresnya Jokowi di ...
4398,1,"5. masa depannya nggak jelas. lha iya, gimana ..."
4399,0,[USERNAME] dulu beneran ada mahasiswa Teknik U...


In [11]:
with pd.option_context('display.max_colwidth', None):
    display(pd.DataFrame({"tweet": x_train, "label": y_train}))

NameError: name 'x_train' is not defined

In [24]:
from sklearn.model_selection import train_test_split

X = data_labelled['tweet']
y = data_labelled['label']

X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
combined_X_train = pd.concat([snorkel_data_20['tweet'], X_train_normal])


In [63]:
combined_y_train = pd.concat([snorkel_data_20['label'], y_train_normal])


In [ ]:
combined_y_train.value_counts()

: 

In [64]:
combined_y_train

0       2
1       0
2       4
3       4
4       0
       ..
3444    0
466     1
3092    2
3772    1
860     2
Name: label, Length: 6820, dtype: int64

In [ ]:
combined_X_train.to_csv('D:\Downloads\Combined_X_train.csv')


In [ ]:
combined_y_train.to_csv('D:\Downloads\Combined_y_train.csv')

In [20]:
data_labelled['tweet'].count()

4401

In [55]:
records = label_model.predict()
label2int = {'happy': 0, 'anger': 1, 'sadness': 2, 'fear': 3, 'love': 4}
x_train = [rec.text for rec in records]
y_train = [label2int[rec.prediction[0][0]] for rec in records]

In [56]:
with pd.option_context('display.max_colwidth', None):
    display(pd.DataFrame({"tweet": x_train, "label": y_train}))



,tweet,label
0,Wihh ikutan happy Congrats sista,0
1,Di marah marahin terus sama bagol padahal sepanjang pertandingan,1
2,Jangan sedih cantik,2
3,Masih syukur UMR Saya dibawah umur Cuman Dan kasihan lagi yang dibawah saya ORANG KAYA ADALAH ORANG SELALU MERASA CUKUP ORANG MISKIN ADALAH ORANG YANG SELALU MERASA KURANG INGAT SAAT NANTI NAFAS STOP DARI BADANMU APA YANG AKAN KAMU PERBUAT DENGAN HARTAMU INGAT MATI,2
4,Happy easter,0
...,...,...
3255,Orang takut mah ngumpett ini takut malah pengen bj Takut aja terus selagi nonton dah wkwkk,3
3256,Patah hati terbesar personil member bukan sebatas memutuskan keluar dari team tp kehilangan member dgn cara menyedihkan dan LBH miris lagi saat dia membutuhkan sosok disamping dia ngerasa sendiri sampai akhir hayatnya ahhh andai waktu bs diputar mungkin cerita akan beda,2
3257,Gacocok jadi influencer baru difollo banyak cowo aja bawaannya malah takut,3
3258,Puasa dibikin kesel emg ada aja da ampun,1


In [52]:
df=pd.DataFrame({"tweet": x_train, "label": y_train})
df.to_csv("D:\Downloads\Snorkel_dataset_with_20_test.csv", encoding='utf-8')


In [14]:
snorkel_data = pd.read_csv("D:\Downloads\Snorkel_dataset_v7.csv")
# snorkel_data['label'] = snorkel_data['label'].replace([0, 1, 2, 3, 4], ['happy', 'anger', 'sadness', 'fear', 'love'] )
snorkel_data.head(1000)

,Unnamed: 0,tweet,label
0,0,org org knpa pada jahat sama gw hari ini astaga,2
1,1,Happy Birthday,0
2,2,ShaniJKT aamiin semangat sayangku bimbingan ad...,4
3,3,Rykarl bucin sangat smp suruh Fir minta maaf k...,4
4,4,funsized Happy birthday,0
...,...,...,...
995,995,Sekarang liat yang lucu dikit aja udah mulai k...,0
996,996,The boss baby How to train your dragon The emo...,0
997,997,HASIL PENGELUARAN HUAHIN Hari Ini SABTU April ...,1
998,998,kalau boleh soalnya takut kekampus sendiri a...,3


In [54]:
snorkel_data_20 = pd.read_csv("D:\Downloads\Snorkel_dataset_with_20_test.csv")
# snorkel_data['label'] = snorkel_data['label'].replace([0, 1, 2, 3, 4], ['happy', 'anger', 'sadness', 'fear', 'love'] )
snorkel_data_20

,Unnamed: 0,tweet,label
0,0,org org knpa pada jahat sama gw hari ini astaga,2
1,1,Happy Birthday,0
2,2,ShaniJKT aamiin semangat sayangku bimbingan ad...,4
3,3,Rykarl bucin sangat smp suruh Fir minta maaf k...,4
4,4,funsized Happy birthday,0
...,...,...,...
3295,3295,Happy birthday boss,0
3296,3296,Yeayy Alhamdulillah lazada pecah telor today s...,0
3297,3297,kaget lebih ke takut si,3
3298,3298,Astaga lupa,2


In [15]:
snorkel_data.drop(columns=['Unnamed: 0'])

,tweet,label
0,org org knpa pada jahat sama gw hari ini astaga,2
1,Happy Birthday,0
2,ShaniJKT aamiin semangat sayangku bimbingan ad...,4
3,Rykarl bucin sangat smp suruh Fir minta maaf k...,4
4,funsized Happy birthday,0
...,...,...
3295,Happy birthday boss,0
3296,Yeayy Alhamdulillah lazada pecah telor today s...,0
3297,kaget lebih ke takut si,3
3298,Astaga lupa,2


In [73]:
snorkel_data_20['label'].count()

3300

In [17]:
from sklearn.model_selection import train_test_split


train_data_snorkel, test_data_snorkel = train_test_split(snorkel_data, test_size=0.2, random_state=42)
train_data_snorkel['label'].value_counts()

0    942
4    553
1    437
3    371
2    337
Name: label, dtype: int64

In [71]:
testing_data['label'].value_counts()

1    226
2    213
0    195
4    124
3    122
Name: label, dtype: int64

In [65]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from transformers import AutoTokenizer, TFAutoModel
# from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import svm

# tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")
# model = TFAutoModel.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7", from_pt=True)

classifier = Pipeline([
    ('vect', CountVectorizer()),
    # ('tfidf', TfidfVectorizer(tokenizer=tokenizer.tokenize)),
    ('clf', svm.SVC(kernel='linear'))
])

classifier.fit(X=combined_X_train, y=combined_y_train,)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', SVC(kernel='linear'))])

In [59]:
test_ds = weak_labels.records(has_annotation=True)


X_test = [rec.text for rec in test_ds]
y_test = [label2int[rec.annotation] for rec in test_ds]

In [66]:
accuracy = classifier.score(
    X=X_test_normal,
    y=y_test_normal,
)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.6140749148694665


In [67]:
from sklearn import metrics

# get predictions for the test set
predicted = classifier.predict(X_test_normal)

print(metrics.classification_report(y_test_normal, predicted, target_names=label2int.keys()))

              precision    recall  f1-score   support

       happy       0.57      0.58      0.58       214
       anger       0.62      0.67      0.65       229
     sadness       0.49      0.48      0.48       200
        fear       0.74      0.62      0.68       119
        love       0.76      0.78      0.77       119

    accuracy                           0.61       881
   macro avg       0.64      0.63      0.63       881
weighted avg       0.62      0.61      0.61       881

